<a href="https://colab.research.google.com/github/enelene/ML-assignment-1/blob/main/model_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of required libraries and env variables


In [5]:
!pip install mlflow dagshub kaggle xgboost scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import mlflow
import os
from dagshub import dagshub_logger
import pandas as pd
import numpy as np
import pickle
from google.colab import drive

In [7]:
os.environ["MLFLOW_TRACKING_USERNAME"] = "enelene"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "8gW@uZ@KYuCwFuU"

In [8]:
mlflow.set_tracking_uri("https://dagshub.com/enelene/ML-assignment-1.mlflow")
mlflow.set_experiment("house-prices-experiment")

<Experiment: artifact_location='mlflow-artifacts:/17fe90981cbc4a7aaeaf700c3b4fd397', creation_time=1743927842431, experiment_id='0', last_update_time=1743927842431, lifecycle_stage='active', name='house-prices-experiment', tags={}>

Importing Data from kaggle

In [9]:
drive.mount('/content/drive')

!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle config view

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Configuration values from /root/.kaggle
- username: elenegabeskiria
- path: None
- proxy: None
- competition: None


Libraries import

In [10]:
# ML libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# MLflow setup
import mlflow.sklearn

Data loading

In [50]:
# For example:
train_path = "/content/drive/MyDrive/kaggle/house_prices/train.csv"
test_path = "/content/drive/MyDrive/kaggle/house_prices/test.csv"

import pandas as pd

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [51]:
# Keep a backup just in case
train_original = train_df.copy()
test_original = test_df.copy()

In [52]:
train_df.info()
train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [53]:
print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)

Train shape: (1460, 81)
Test shape : (1459, 80)


Helper functions

In [54]:
def fill_missing_values(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('None')
        elif pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(0)
    return df

def show_missing_values(df):
    missing_values = df.isnull().sum()
    missing_values_percentage = (missing_values / len(df)) * 100
    missing_data = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_values_percentage})
    missing_data = missing_data[missing_data['Missing Values'] > 0]
    missing_data = missing_data.sort_values('Percentage', ascending=False)
    return missing_data


In [55]:
print(show_missing_values(test_original))
print(show_missing_values(train_original))

              Missing Values  Percentage
PoolQC                  1456   99.794380
MiscFeature             1408   96.504455
Alley                   1352   92.666210
Fence                   1169   80.123372
MasVnrType               894   61.274846
FireplaceQu              730   50.034270
LotFrontage              227   15.558602
GarageQual                78    5.346127
GarageCond                78    5.346127
GarageYrBlt               78    5.346127
GarageFinish              78    5.346127
GarageType                76    5.209047
BsmtCond                  45    3.084304
BsmtQual                  44    3.015764
BsmtExposure              44    3.015764
BsmtFinType1              42    2.878684
BsmtFinType2              42    2.878684
MasVnrArea                15    1.028101
MSZoning                   4    0.274160
Functional                 2    0.137080
BsmtFullBath               2    0.137080
Utilities                  2    0.137080
BsmtHalfBath               2    0.137080
Exterior1st     

In [56]:
train_df = fill_missing_values(train_df)
test_df = fill_missing_values(test_df)

print("Missing values in train dataset:")
print(show_missing_values(train_df))
print("\nMissing values in test dataset:")
print(show_missing_values(test_df))

Missing values in train dataset:
Empty DataFrame
Columns: [Missing Values, Percentage]
Index: []

Missing values in test dataset:
Empty DataFrame
Columns: [Missing Values, Percentage]
Index: []


For Randomness

In [57]:
SEED = 42
np.random.seed(SEED)

Simple Feature Engineering at first

In [58]:
train_df["TotalSF"] = train_df["TotalBsmtSF"] + train_df["1stFlrSF"] + train_df["2ndFlrSF"]
test_df["TotalSF"] = test_df["TotalBsmtSF"] + test_df["1stFlrSF"] + test_df["2ndFlrSF"]


In [20]:
y = np.log1p(train_df["SalePrice"])

Dont need these columns for training

In [21]:
train_df.drop(["Id","SalePrice"], axis=1, inplace=True)
test_ids = test_df["Id"]
test_df.drop("Id", axis=1, inplace=True)

Generation of dummy columns

In [22]:
all_data = pd.concat([train_df, test_df], axis=0)  # Combine to ensure same dummy columns
all_data = pd.get_dummies(all_data, drop_first=True)


In [23]:
train_transformed = all_data.iloc[:len(train_df), :]
test_transformed = all_data.iloc[len(train_df):, :]

In [24]:
X = train_transformed.values
X_test = test_transformed.values

In [25]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

For MLFlow tracking

In [65]:
def train_and_log(model, model_name, params=None):
    with mlflow.start_run(run_name=model_name):
        if params:
            for p_name, p_val in params.items():
                mlflow.log_param(p_name, p_val)

        # Fit and evaluate
        model.fit(X_train, y_train)
        preds_val = model.predict(X_val)
        rmse_val = np.sqrt(mean_squared_error(y_val, preds_val))

        mlflow.log_metric("rmse_val", rmse_val)


        # Log model artifact
        mlflow.sklearn.log_model(model, artifact_path="model")

        print(f"{model_name} RMSE (Val): {rmse_val:.4f}")
        return rmse_val, model

Simple models

In [ ]:
#Linear Regression (Baseline)
linreg_params = {}
train_and_log(LinearRegression(), "LinearRegression", linreg_params)

#Random Forest
rf_params = {"n_estimators": 100, "max_depth": 10}
rf_model = RandomForestRegressor(
    n_estimators=rf_params["n_estimators"],
    max_depth=rf_params["max_depth"],
    random_state=SEED
)
train_and_log(rf_model, "RandomForest", rf_params)

2025/04/09 07:45:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LinearRegression RMSE (Val): 0.1734
🏃 View run LinearRegression at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/c6262b87208f45efaefd25decf7d2688
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


2025/04/09 07:45:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest RMSE (Val): 0.1508
🏃 View run RandomForest at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/4d08c49fede34668a668d46b5dec13f3
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


Finding best hyperparams with RandomForestRegressor

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


rf = RandomForestRegressor(random_state=SEED)

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, 15, None]
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",  # scikit-learn uses negative MSE for max objective
    cv=5,                              # 5 folds
    verbose=2,                         # print logs while training (optional)
    n_jobs=-1                          # use all available CPU cores
)

grid_search.fit(X_train, y_train)

print("Best params found:", grid_search.best_params_)
print("Best CV score:", grid_search.best_score_)

best_rf = grid_search.best_estimator_

y_val_preds = best_rf.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_preds))

print("Validation RMSE with best params:", val_rmse)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best params found: {'max_depth': None, 'n_estimators': 200}
Best CV score: -0.021024343017266335
Validation RMSE with best params: 0.14754298333238972


Finding hyperparams with RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    "n_estimators": randint(50, 500),
    "max_depth": [5, 10, 15, None]
}

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    scoring="neg_mean_squared_error",
    cv=5,
    n_iter=10,  # how many random combos to try
    random_state=SEED
)
random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_depth': [5, 10, 15, None],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a463c56dfd0>},
                   random_state=42, scoring='neg_mean_squared_error')

Using above two results to optimize previous models

In [ ]:
rf_params = {"n_estimators": 200, "max_depth": None }
rf_model = RandomForestRegressor(
    n_estimators=rf_params["n_estimators"],
    max_depth=rf_params["max_depth"],
    random_state=SEED
)
train_and_log(rf_model, "RandomForest", rf_params)

2025/04/09 08:55:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest RMSE (Val): 0.1475
🏃 View run RandomForest at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/b07239451908434293e3dc3b97a9479a
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


In [ ]:
rf_params = {"n_estimators": 485, "max_depth": 15}
rf_model = RandomForestRegressor(
    n_estimators=rf_params["n_estimators"],
    max_depth=rf_params["max_depth"],
    random_state=SEED
)
train_and_log(rf_model, "RandomForest", rf_params)

2025/04/09 08:58:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest RMSE (Val): 0.1481
🏃 View run RandomForest at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/0deda8dbaa564e2db0f899600e8f9254
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


Advanced Feature Engineering

In [47]:
train_df = train_original
test_df = test_original

train_df = fill_missing_values(train_df)
test_df = fill_missing_values(test_df)

In [59]:
outlier_threshold = 4000
mask = train_df["GrLivArea"] < outlier_threshold
train_df = train_df[mask]  # remove outlier rows

train_df["TotalSF"] = train_df["TotalBsmtSF"] + train_df["1stFlrSF"] + train_df["2ndFlrSF"]
test_df["TotalSF"] = test_df["TotalBsmtSF"] + test_df["1stFlrSF"] + test_df["2ndFlrSF"]

train_df["OverallQual_TotSF"] = train_df["OverallQual"] * (
    train_df["TotalBsmtSF"] + train_df["1stFlrSF"] + train_df["2ndFlrSF"]
)
test_df["OverallQual_TotSF"] = test_df["OverallQual"] * (
    test_df["TotalBsmtSF"] + test_df["1stFlrSF"] + test_df["2ndFlrSF"]
)
# Age of the house when sold
train_df['House_Age'] = train_df['YrSold'] - train_df['YearBuilt']
test_df['House_Age'] = test_df['YrSold'] - test_df['YearBuilt']

# Years since renovation
train_df['Years_Since_Remod'] = train_df['YrSold'] - train_df['YearRemodAdd']
test_df['Years_Since_Remod'] = test_df['YrSold'] - test_df['YearRemodAdd']

# Whether house was remodeled
train_df['Was_Remodeled'] = (train_df['YearRemodAdd'] > train_df['YearBuilt']).astype(int)
test_df['Was_Remodeled'] = (test_df['YearRemodAdd'] > test_df['YearBuilt']).astype(int)

# Total bathrooms (combining full and half baths)
train_df['TotalBaths'] = train_df['FullBath'] + (0.5 * train_df['HalfBath']) + train_df['BsmtFullBath'] + (0.5 * train_df['BsmtHalfBath'])
test_df['TotalBaths'] = test_df['FullBath'] + (0.5 * test_df['HalfBath']) + test_df['BsmtFullBath'] + (0.5 * test_df['BsmtHalfBath'])

# Living area per bedroom
train_df['LivingAreaPerBedroom'] = train_df['GrLivArea'] / (train_df['BedroomAbvGr'] + 1)  # +1 to avoid division by zero
test_df['LivingAreaPerBedroom'] = test_df['GrLivArea'] / (test_df['BedroomAbvGr'] + 1)

# Lot ratio (useful for understanding space utilization)
train_df['LotRatio'] = train_df['1stFlrSF'] / train_df['LotArea']
test_df['LotRatio'] = test_df['1stFlrSF'] / test_df['LotArea']

# Total porch area
train_df['TotalPorchSF'] = train_df['OpenPorchSF'] + train_df['EnclosedPorch'] + train_df['3SsnPorch'] + train_df['ScreenPorch']
test_df['TotalPorchSF'] = test_df['OpenPorchSF'] + test_df['EnclosedPorch'] + test_df['3SsnPorch'] + test_df['ScreenPorch']

# Overall quality and condition interaction
train_df['Quality_x_Condition'] = train_df['OverallQual'] * train_df['OverallCond']
test_df['Quality_x_Condition'] = test_df['OverallQual'] * test_df['OverallCond']

# Kitchen quality with area
train_df['KitchenQual_Encoded'] = train_df['KitchenQual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0})
test_df['KitchenQual_Encoded'] = test_df['KitchenQual'].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0})
train_df['Kitchen_Quality_SF'] = train_df['KitchenQual_Encoded'] * train_df['1stFlrSF']
test_df['Kitchen_Quality_SF'] = test_df['KitchenQual_Encoded'] * test_df['1stFlrSF']

# Create neighborhood price level feature (requires target encoding with cross-validation)
neighborhood_means = train_df.groupby('Neighborhood')['SalePrice'].mean()
train_df['Neighborhood_Price_Level'] = train_df['Neighborhood'].map(neighborhood_means)
test_df['Neighborhood_Price_Level'] = test_df['Neighborhood'].map(neighborhood_means)

# Location quality (OverallQual * Neighborhood median price)
train_df['Location_Quality'] = train_df['OverallQual'] * train_df['Neighborhood_Price_Level']
test_df['Location_Quality'] = test_df['OverallQual'] * test_df['Neighborhood_Price_Level']

# Create boolean features for columns with significant missing values
for col in ['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']:
    train_df[f'Has_{col}'] = (~train_df[col].isin(['None'])).astype(int)
    test_df[f'Has_{col}'] = (~test_df[col].isin(['None'])).astype(int)

# Total finished basement area percentage
train_df['TotalFinishedBsmtSF'] = train_df['BsmtFinSF1'] + train_df['BsmtFinSF2']
test_df['TotalFinishedBsmtSF'] = test_df['BsmtFinSF1'] + test_df['BsmtFinSF2']
train_df['FinishedBsmtRatio'] = train_df['TotalFinishedBsmtSF'] / (train_df['TotalBsmtSF'] + 1)
test_df['FinishedBsmtRatio'] = test_df['TotalFinishedBsmtSF'] / (test_df['TotalBsmtSF'] + 1)

# Garage age when sold
train_df['Garage_Age'] = train_df['YrSold'] - train_df['GarageYrBlt']
test_df['Garage_Age'] = test_df['YrSold'] - test_df['GarageYrBlt']
# Replace negative ages with 0 (garage built after house)
train_df['Garage_Age'] = train_df['Garage_Age'].clip(lower=0)
test_df['Garage_Age'] = test_df['Garage_Age'].clip(lower=0)

# Garage area per car
train_df['GarageArea_Per_Car'] = train_df['GarageArea'] / (train_df['GarageCars'] + 0.1)  # Avoid division by zero
test_df['GarageArea_Per_Car'] = test_df['GarageArea'] / (test_df['GarageCars'] + 0.1)


# Square footage squared (captures non-linear relationship with price)
train_df['GrLivArea_Sq'] = train_df['GrLivArea'] ** 2
test_df['GrLivArea_Sq'] = test_df['GrLivArea'] ** 2

# Interaction between house size and overall quality
train_df['GrLivArea_x_Quality'] = train_df['GrLivArea'] * train_df['OverallQual']
test_df['GrLivArea_x_Quality'] = test_df['GrLivArea'] * test_df['OverallQual']

# Age and quality interaction
train_df['Age_x_Quality'] = train_df['House_Age'] * train_df['OverallQual']
test_df['Age_x_Quality'] = test_df['House_Age'] * test_df['OverallQual']

# Season sold
train_df['Season_Sold'] = pd.cut(
    train_df['MoSold'],
    bins=[0, 3, 6, 9, 12],
    labels=['Winter', 'Spring', 'Summer', 'Fall'],
    include_lowest=True
)
test_df['Season_Sold'] = pd.cut(
    test_df['MoSold'],
    bins=[0, 3, 6, 9, 12],
    labels=['Winter', 'Spring', 'Summer', 'Fall'],
    include_lowest=True
)
# Convert to dummies (handled by get_dummies later)

<ipython-input-59-15df8ab1cba4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["TotalSF"] = train_df["TotalBsmtSF"] + train_df["1stFlrSF"] + train_df["2ndFlrSF"]
<ipython-input-59-15df8ab1cba4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["OverallQual_TotSF"] = train_df["OverallQual"] * (
<ipython-input-59-15df8ab1cba4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [60]:
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

test_ids = test_df["Id"]
train_df.drop(["Id"], axis=1, inplace=True)
test_df.drop(["Id"], axis=1, inplace=True)

all_data = pd.concat([train_df.drop("SalePrice", axis=1), test_df], axis=0)
all_data = pd.get_dummies(all_data, drop_first=True)

train_transformed = all_data.iloc[: len(train_df), :]
test_transformed = all_data.iloc[len(train_df) :, :]

X = train_transformed.values
y = train_df["SalePrice"].values
X_test = test_transformed.values

train_columns = train_transformed.columns

In [61]:
import pickle
with open("/content/drive/MyDrive/kaggle/house_prices/train_columns.pkl", "wb") as f:
    pickle.dump(train_columns, f)


In [62]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

same stuff here + XGBRegressor

In [63]:
from xgboost import XGBRegressor
rf = RandomForestRegressor(random_state=SEED)

param_grid_rf = {
    "n_estimators": [100, 200],
    "max_depth": [10, 15, None],
}

grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring="neg_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)

print("Fitting RandomForest GridSearch...")
grid_rf.fit(X_train, y_train)

best_params_rf = grid_rf.best_params_
print("Best RandomForest params:", best_params_rf)
best_rf_model = grid_rf.best_estimator_


Fitting RandomForest GridSearch...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best RandomForest params: {'max_depth': 15, 'n_estimators': 200}


In [66]:
rf_rmse_val, best_rf_model = train_and_log(
    best_rf_model,
    "RandomForest_CV",
    best_params_rf
)


2025/04/10 12:36:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest_CV RMSE (Val): 0.1453
🏃 View run RandomForest_CV at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/b4b20dacf6044fe4b9f6ab424ee3d51f
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


In [67]:
#XGBoost Hyperparam Tuning via GridSearchCV
xgb = XGBRegressor(random_state=SEED, use_label_encoder=False)

param_grid_xgb = {
    "n_estimators": [200, 300],
    "max_depth": [3, 5],
    "learning_rate": [0.05, 0.1],
}

grid_xgb = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid_xgb,
    scoring="neg_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)

In [68]:

print("Fitting XGBoost GridSearch...")
grid_xgb.fit(X_train, y_train, verbose=False)

best_params_xgb = grid_xgb.best_params_
print("Best XGBoost params:", best_params_xgb)
best_xgb_model = grid_xgb.best_estimator_

xgb_rmse_val, best_xgb_model = train_and_log(
    best_xgb_model,
    "XGBoost_CV",
    best_params_xgb
)

Fitting XGBoost GridSearch...
Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:36:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best XGBoost params: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:36:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2025/04/10 12:36:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


XGBoost_CV RMSE (Val): 0.1347
🏃 View run XGBoost_CV at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/0f1ec2c4fe5b45009e1ed0516e914600
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0


ENSEMBLING - trying to average predictions from best RF + best XGB

In [69]:
def ensemble_predict(models, weights, X_input):
    """
    Weighted average of predictions from two models.
    models: list of fitted models
    weights: same length list of floats
    X_input: feature matrix
    """
    assert len(models) == len(weights), "models and weights must have same length"
    preds = [m.predict(X_input) * w for m, w in zip(models, weights)]
    return np.sum(preds, axis=0) / np.sum(weights)

In [70]:
models_list = [best_rf_model, best_xgb_model]
weights_list = [1.0, 1.0]  # each model is weighted equally

ensemble_preds_val = ensemble_predict(models_list, weights_list, X_val)
ensemble_rmse_val = np.sqrt(mean_squared_error(y_val, ensemble_preds_val))

with mlflow.start_run(run_name="Ensemble_RF_XGB"):
    mlflow.log_param("ensemble_method", "average")
    mlflow.log_param("weights", "equal")
    mlflow.log_metric("rmse_val", ensemble_rmse_val)
    print(f"[Ensemble_RF_XGB] RMSE (Val): {ensemble_rmse_val:.4f}")

[Ensemble_RF_XGB] RMSE (Val): 0.1366
🏃 View run Ensemble_RF_XGB at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0/runs/e80af723993944569650a193ceb22918
🧪 View experiment at: https://dagshub.com/enelene/ML-assignment-1.mlflow/#/experiments/0
